In [2]:


import pandas as pd
import nameparser
import json
import requests
import numpy as np

/Users/mats_dodd/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
pro_publica_api_key = "zkptMlkDXn3oi2IJq8g3EYU5JE9VCGaVNGP3ud5H"

senate_url = f"https://api.propublica.org/congress/v1/118/senate/members.json"
headers = {'X-API-Key': pro_publica_api_key}
senate = requests.get(senate_url, headers=headers)

senate.json()

{'status': 'OK',
 'copyright': ' Copyright (c) 2023 Pro Publica Inc. All Rights Reserved.',
 'results': [{'congress': '118',
   'chamber': 'Senate',
   'num_results': 102,
   'offset': 0,
   'members': [{'id': 'B001230',
     'title': 'Senator, 1st Class',
     'short_title': 'Sen.',
     'api_uri': 'https://api.propublica.org/congress/v1/members/B001230.json',
     'first_name': 'Tammy',
     'middle_name': None,
     'last_name': 'Baldwin',
     'suffix': None,
     'date_of_birth': '1962-02-11',
     'gender': 'F',
     'party': 'D',
     'leadership_role': 'Senate Democratic Caucus Secretary',
     'twitter_account': 'SenatorBaldwin',
     'facebook_account': 'senatortammybaldwin',
     'youtube_account': 'witammybaldwin',
     'govtrack_id': '400013',
     'cspan_id': '57884',
     'votesmart_id': '3470',
     'icpsr_id': '29940',
     'crp_id': 'N00004367',
     'google_entity_id': '/m/024v02',
     'fec_candidate_id': 'H8WI00018',
     'url': 'https://www.baldwin.senate.gov',
  

In [4]:
url = f"https://api.propublica.org/congress/v1/118/Senate/committees/HSFA/subcommittees/HSFA16.json"
r = requests.get(url, headers=headers)
r.json()

{'status': '500', 'error': 'Internal Server Error'}

In [ ]:
"""Get all comittees and sub comittees for 118th congress"""

url_senate_118 = "https://api.propublica.org/congress/v1/118/senate/committees.json"
url_house_118 = "https://api.propublica.org/congress/v1/118/house/committees.json"
url_joint_118 = "https://api.propublica.org/congress/v1/118/joint/committees.json"

headers = {'X-API-Key': pro_publica_api_key}
senate_comittees_118 = requests.get(url_senate_118, headers=headers)
house_comittees_118 = requests.get(url_house_118, headers=headers)
joint_comittees_118 = requests.get(url_joint_118, headers=headers)


r_sen_118 = senate_comittees_118.json()['results'][0]
r_house_118 = house_comittees_118.json()['results'][0]
r_joint_118 = joint_comittees_118.json()['results'][0]

id = []
name = []
chamber = []
subcomittee = {}
subcomittee['id'] = []
subcomittee['name'] = []
subcomittee['parent_id'] = []

for comittee in r_sen_118['committees']:
    id.append(comittee['id'])
    name.append(comittee['name'])
    chamber.append(comittee['chamber'])
    for sub_comittee in comittee['subcommittees']:
        subcomittee['parent_id'].append(comittee['id'])
        subcomittee['id'].append(sub_comittee['id'])
        subcomittee['name'].append(sub_comittee['name'])

for comittee in r_house_118['committees']:
    id.append(comittee['id'])
    name.append(comittee['name'])
    chamber.append(comittee['chamber'])
    for sub_comittee in comittee['subcommittees']:
        subcomittee['parent_id'].append(comittee['id'])
        subcomittee['id'].append(sub_comittee['id'])
        subcomittee['name'].append(sub_comittee['name'])

for comittee in r_joint_118['committees']:
    id.append(comittee['id'])
    name.append(comittee['name'])
    chamber.append(comittee['chamber'])
    for sub_comittee in comittee['subcommittees']:
        subcomittee['parent_id'].append(comittee['id'])
        subcomittee['id'].append(sub_comittee['id'])
        subcomittee['name'].append(sub_comittee['name'])

subcomittees_118 = pd.DataFrame(subcomittee)
comittees_118 = pd.DataFrame({'id': id, 'name': name, 'chamber': chamber})
committee_subcommittee_118 = subcomittees_118.merge(comittees_118,
                                                    left_on='parent_id',
                                                    right_on='id',
                                                    how='left', ).rename(columns={'name_x': 'subcomittee_name',
                                                                                  'id_x': 'subcomittee_id',
                                                                                  'parent_id': 'committee_id',
                                                                                  'name_y': 'comittee_name'}).drop(columns=['id_y']).rename(columns={'id_x': 'id'})[['comittee_name',
                                                                                                                                                                     'subcomittee_name',
                                                                                                                                                                     'committee_id',
                                                                                                                                                                     'subcomittee_id',
                                                                                                                                                                     'chamber']]

committee_subcommittee_118

In [ ]:
"""Get members of all committees ascociated with senate in 118th congress"""

dfs = []
comittee_list_senate = comittees_118[comittees_118['chamber']=='Senate']['id'].unique()
for committee_id in comittee_list_senate:
    url = f"https://api.propublica.org/congress/v1/118/senate/committees/{committee_id}.json"
    r = requests.get(url, headers=headers)
    members = pd.DataFrame(r.json()['results'][0]['current_members'])[['id', 'name']]
    members['comittee_id'] = committee_id
    dfs.append(members)

comittee_list_house = comittees_118[comittees_118['chamber']=='House']['id'].unique()
for committee_id in comittee_list_house:
    url = f"https://api.propublica.org/congress/v1/118/house/committees/{committee_id}.json"
    r = requests.get(url, headers=headers)
    members = pd.DataFrame(r.json()['results'][0]['current_members'])[['id', 'name']]
    members['comittee_id'] = committee_id
    dfs.append(members)

comittee_list_joint = comittees_118[comittees_118['chamber']=='Joint']['id'].unique()
for committee_id in comittee_list_joint:
    url = f"https://api.propublica.org/congress/v1/118/joint/committees/{committee_id}.json"
    r = requests.get(url, headers=headers)
    members = pd.DataFrame(r.json()['results'][0]['current_members'])[['id', 'name']]
    members['comittee_id'] = committee_id
    dfs.append(members)


committee_membership_118 = pd.concat(dfs)

committee_membership_118 = pd.get_dummies(committee_membership_118, columns=['comittee_id'], prefix='', prefix_sep='').groupby(['id', 'name']).sum().reset_index()

committee_membership_118